In [ ]:
### SVM classification on PCA of MNIST and OMNIGLOT

In [ ]:
## SVM classification on PCA of MNIST

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from matplotlib import pyplot as plt

In [ ]:
##Load the training, testing and validation dataset
def load_data():
    train,test,val = np.load('InData/mnist.pkl',mmap_mode='r')
    print 'x_train \t | y_train \t| x_test \t | y_test \t| x_val \t| y_val \n----------------------------------------------------------------------------------------'
    print np.shape(train[0]),'\t',np.shape(train[1]),'\t',np.shape(test[0]),'\t',np.shape(test[1]),'\t',np.shape(val[0]),'\t',np.shape(val[1])
    return train[0], train[1], test[0], test[1], val[0], val[1]
x_train,y_train,x_test,y_test,x_val,y_val=load_data()

In [ ]:
#Perform PCA to get 10 elements
model=PCA(n_components=10).fit(x_train) #
enc_train=model.transform(x_train)
enc_test=model.transform(x_test)
classifier=SVC().fit(enc_train,y_train) #default rbf kernel
score=classifier.score(enc_test,y_test)
print "mean accuracy (SVM):",score*100,'%'


In [ ]:
#Perfom PCA to get 2 elements and visualise
model=PCA(n_components=2).fit(x_train)
enc_test=model.transform(x_test)
colours=['red','green','blue','black','pink','orange','purple','gray','cyan','magenta']
for i in range(enc_test.shape[0]):
        plt.scatter(enc_test[i,0],enc_test[i,1], color=colours[y_test[i]])
        plt.title('Test Data')
plt.show()

In [ ]:
## SVM classification on PCA of OMNIGLOT

In [ ]:
import theano
floatX=theano.config.floatX

In [ ]:
##Load the training, testing and validation datasets
def load_data():
    x_train=np.load('InData/omniglot/omni_back_shfl_X.npy', mmap_mode='r')
    y_train=np.load('InData/omniglot/omni_back_shfl_Y.npy',mmap_mode='r')
    x_test=np.load('InData/omniglot/omni_eval_X.npy',mmap_mode='r')
    y_test=np.load('InData/omniglot/omni_eval_Y.npy', mmap_mode='r')
    
    x_temp=np.load('InData/omniglot/omni_back_X.npy',mmap_mode='r')
    y_temp=np.load('InData/omniglot/omni_back_Y.npy',mmap_mode='r')
    x_val=np.reshape(np.concatenate([x_temp[i] for i in range(0,x_temp.shape[0],20)]),(-1,1,105,105))
    y_val=[y_temp[i] for i in range(0,x_temp.shape[0],20)]
    
    print 'x_train  \t\t | y_train \t| x_test  \t\t| y_test \t  | x_val   \t\t | y_val \n-----------------------------------------------------------------------------------------------------------'
    print np.shape(x_train),'\t',np.shape(y_train),'\t',np.shape(x_test),'\t',np.shape(y_test),'\t\t',np.shape(x_val),'\t',np.shape(y_val)
    return x_train.astype(floatX), y_train, x_test.astype(floatX), y_test, x_val.astype(floatX), y_val

In [ ]:
x_train,y_train,x_test,y_test,x_val,y_val=load_data()

In [ ]:
#Perform PCA to get 200 elements (same as n_hidden used to trian most DAAE, iDAAE, AAE models)
x_train=x_train.reshape(-1,105*105)
x_val=x_val.reshape(-1,105*105)
model=PCA(n_components=200).fit(x_train.reshape(-1,105*105)) #
enc_train=model.transform(x_train)
enc_val=model.transform(x_val)
classifier=SVC().fit(enc_train,y_train) #default rbf kernel
y_val=np.asarray(y_val, dtype=int)
score=classifier.score(enc_val,y_val)
print "mean accuracy (SVM):",score*100,'%'

In [ ]:
## 20 class classfication (for 1st 20 classes of the eval data set using leave one out method)
start=0 # =0 to take 1st 20 (>0 for other subsets - multiple of 20)
model=PCA(n_components=200).fit(x_test[start:start+20*20].reshape(-1,105*105)) #
enc_test=model.transform(x_test[start:start+20*20].reshape(-1,105*105))
print np.shape(enc_test)

all_score_test=[]
all_score_train=[]
for i in range(20): #keep out the ith samples, train on the rest
    x_20_test=enc_test[i::20]; #take the ith samples for testing
    y_20_test=y_test[start:start+20*20][i::20]; #take the ith samples for testing for 1st 20 classes
    x_20_train=np.delete(enc_test, range(i,enc_test.shape[0],20), axis=0) #remove the ith samples for training
    y_20_train=np.delete(y_test[start:start+20*20], range(i,enc_test.shape[0],20), axis=0) #remove the ith samples for training for the 1st 20 classes

#     print 'stand out samples:\n',range(i,enc_test.shape[0],20)

#     print np.unique(y_20_test),'\n', np.unique(y_20_train)
#     print np.shape(x_20_test), np.shape(x_20_train), np.shape(y_20_test), np.shape(y_20_train)

    classifier=SVC(kernel='linear').fit(x_20_train,y_20_train) #default rbf kernel
    y_val=np.asarray(y_val,dtype=int)
    score_test=classifier.score(x_20_test,y_20_test)
    score_train=classifier.score(x_20_train, y_20_train)
    all_score_test.append(score_test)
    all_score_train.append(score_train)
print "\n mean test accuracy on ", len(all_score_test), "sets of 20-way classification:", np.mean(all_score_test)*100,"%"
print "\n (mean train accuracy on ", len(all_score_train), "sets of 20-way classification:", np.mean(all_score_train)*100,"%"
print "\n max test accuracy on ", len(all_score_test), "sets of 20-way classification:", np.max(all_score_test)*100,"%"

In [ ]:
exit()